# Deployment

Various deployment flows using tools

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 26/01/2026   | Martin | Created   | Deployment notebook created | 

# Content

* [Mlflow w/ Docker](#mlflow-w-docker)

# Mlflow w/ Docker

Using Mlflow registry with Docker containers for serving

In [45]:
import os
import mlflow
from dotenv import dotenv_values
config = dotenv_values("../.env")

os.environ["AWS_ACCESS_KEY_ID"] = config["MLFLOW_USER"]
os.environ["AWS_SECRET_ACCESS_KEY"] = config["MLFLOW_PASSWORD"]
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://127.0.0.1:9000"
os.environ["MLFLOW_S3_IGNORE_TLS"] = "true"

mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [46]:
import pickle
import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

from mlflow.models import infer_signature

In [47]:
mlflow.set_experiment("taxi-prediction")

2026/01/26 15:15:31 INFO mlflow.tracking.fluent: Experiment with name 'taxi-prediction' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow/1', creation_time=1769411731192, experiment_id='1', last_update_time=1769411731192, lifecycle_stage='active', name='taxi-prediction', tags={}>

In [48]:
def read_dataframe(filename: str):
  df = pd.read_parquet(filename)

  df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
  df.duration = df.duration.dt.total_seconds() / 60
  df = df[(df.duration >= 1) & (df.duration <= 60)]

  categorical = ['PULocationID', 'DOLocationID']
  df[categorical] = df[categorical].astype(str)
  return df


def prepare_dictionaries(df: pd.DataFrame):
  df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
  categorical = ['PU_DO']
  numerical = ['trip_distance']
  dicts = df[categorical + numerical].to_dict(orient='records')
  return dicts

In [54]:
df_train = read_dataframe('../data/raw/green_tripdata_2025-01.parquet')
df_val = read_dataframe('../data/raw/green_tripdata_2025-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [56]:
with mlflow.start_run():
  params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
  cols = ['PU_DO', 'trip_distance']
  mlflow.log_params(params)

  model = RandomForestRegressor(**params, n_jobs=-1)

  model.fit(df_train[cols], y_train)
  y_pred = model.predict(df_val[cols])

  rmse = mean_squared_error(y_pred, y_val)
  print(params, rmse)
  mlflow.log_metric('rmse', rmse)

  signature = infer_signature(df_train[cols].iloc[[0]], y_train[0])

  mlflow.sklearn.log_model(model, name="duration-pred", input_example=df_train[cols].iloc[[0]], signature=signature)

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 29.065911785392863


🏃 View run chill-koi-886 at: http://127.0.0.1:5000/#/experiments/1/runs/77a718179132447e82f6ea35184e4862
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [58]:
model_load = mlflow.sklearn.load_model("runs:/77a718179132447e82f6ea35184e4862/duration-pred")

In [59]:
df_train.iloc[0]

VendorID                                   2
lpep_pickup_datetime     2025-01-01 00:03:01
lpep_dropoff_datetime    2025-01-01 00:17:12
store_and_fwd_flag                         N
RatecodeID                               1.0
PULocationID                              75
DOLocationID                             235
passenger_count                          1.0
trip_distance                           5.93
fare_amount                             24.7
extra                                    1.0
mta_tax                                  0.5
tip_amount                               6.8
tolls_amount                             0.0
ehail_fee                                NaN
improvement_surcharge                    1.0
total_amount                            34.0
payment_type                             1.0
trip_type                                1.0
congestion_surcharge                     0.0
cbd_congestion_fee                       0.0
duration                           14.183333
PU_DO     

In [60]:
model.predict(df_val[cols].iloc[[0]])

array([5.17751339])

In [44]:
df_val[cols].iloc[[0]]

,PU_DO,trip_distance
0,166_41,0.65


In [ ]:
%load_ext watermark
%watermark